In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [10]:
np.random.seed(1)
a = np.random.random_integers( 0 , 1, size = (7, 7) )
a

array([[1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 1]])

In [11]:
conv_filter = np.random.random_integers( 0 , 1, size = (3, 3) )
print(conv_filter)
print(conv_filter.shape)

[[0 0 1]
 [0 0 1]
 [1 1 0]]
(3, 3)


In [12]:
# generating sub-matrices using stride 1
sub_shape = (3,3)
view_shape = tuple(np.subtract(a.shape, sub_shape) + 1) + sub_shape
strides = a.strides + a.strides

sub_matrices = np.lib.stride_tricks.as_strided(a,view_shape,strides)
print(sub_matrices)
print(sub_matrices.shape)

[[[[1 1 0]
   [1 1 0]
   [1 0 0]]

  [[1 0 0]
   [1 0 0]
   [0 0 1]]

  [[0 0 1]
   [0 0 1]
   [0 1 0]]

  [[0 1 1]
   [0 1 0]
   [1 0 0]]

  [[1 1 1]
   [1 0 1]
   [0 0 0]]]


 [[[1 1 0]
   [1 0 0]
   [1 0 0]]

  [[1 0 0]
   [0 0 1]
   [0 0 1]]

  [[0 0 1]
   [0 1 0]
   [0 1 0]]

  [[0 1 0]
   [1 0 0]
   [1 0 0]]

  [[1 0 1]
   [0 0 0]
   [0 0 0]]]


 [[[1 0 0]
   [1 0 0]
   [1 0 0]]

  [[0 0 1]
   [0 0 1]
   [0 0 0]]

  [[0 1 0]
   [0 1 0]
   [0 0 1]]

  [[1 0 0]
   [1 0 0]
   [0 1 1]]

  [[0 0 0]
   [0 0 0]
   [1 1 1]]]


 [[[1 0 0]
   [1 0 0]
   [1 1 0]]

  [[0 0 1]
   [0 0 0]
   [1 0 0]]

  [[0 1 0]
   [0 0 1]
   [0 0 0]]

  [[1 0 0]
   [0 1 1]
   [0 0 1]]

  [[0 0 0]
   [1 1 1]
   [0 1 1]]]


 [[[1 0 0]
   [1 1 0]
   [1 1 1]]

  [[0 0 0]
   [1 0 0]
   [1 1 1]]

  [[0 0 1]
   [0 0 0]
   [1 1 0]]

  [[0 1 1]
   [0 0 1]
   [1 0 1]]

  [[1 1 1]
   [0 1 1]
   [0 1 1]]]]
(5, 5, 3, 3)


In [13]:
# convolution operation
m = np.einsum('ij,klij->kl',conv_filter,sub_matrices)
print(m)

[[1 0 3 2 2]
 [1 1 2 1 1]
 [1 2 0 1 2]
 [2 2 1 1 2]
 [2 2 3 3 3]]


In [0]:
'''
    2D Pooling function

    Parameters:
        A: input 2D array
        kernel_size: int, the size of the window
        stride: int, the stride of the window
        padding: int, implicit zero paddings on both sides of the input
        pool_mode: string, 'max' or 'avg'
'''
from numpy.lib.stride_tricks import as_strided

def pool2d(A, kernel_size, stride, padding, pool_mode='max'):
    # Padding
    A = np.pad(A, padding, mode='constant')

    # Window view of A
    output_shape = ((A.shape[0] - kernel_size)//stride + 1,
                    (A.shape[1] - kernel_size)//stride + 1)
    kernel_size = (kernel_size, kernel_size)
    A_w = as_strided(A, shape = output_shape + kernel_size, 
                        strides = (stride*A.strides[0],
                                   stride*A.strides[1]) + A.strides)
    A_w = A_w.reshape(-1, *kernel_size)

    # Return the result of pooling
    if pool_mode == 'max':
        return A_w.max(axis=(1,2)).reshape(output_shape)
    elif pool_mode == 'avg':
        return A_w.mean(axis=(1,2)).reshape(output_shape)

In [15]:
# max-pooling
pool2d(m, kernel_size=2, stride=1, padding=0, pool_mode='max')

array([[1, 3, 3, 2],
       [2, 2, 2, 2],
       [2, 2, 1, 2],
       [2, 3, 3, 3]])

In [16]:
# average-pooling
pool2d(m, kernel_size=2, stride=1, padding=0, pool_mode='avg')

array([[0.75, 1.5 , 2.  , 1.5 ],
       [1.25, 1.25, 1.  , 1.25],
       [1.75, 1.25, 0.75, 1.5 ],
       [2.  , 2.  , 2.  , 2.25]])